In [1]:
import torch

def check_devices():
    print("PyTorch version:", torch.__version__)
    print("\nAvailable devices:")
    if torch.cuda.is_available():
        print("CUDA is available!")
        print("Number of CUDA devices:", torch.cuda.device_count())
        for i in range(torch.cuda.device_count()):
            print(f"Device {i}: {torch.cuda.get_device_name(i)}")
        print("\nCurrent CUDA device:", torch.cuda.current_device())
        print("Current device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
    else:
        print("CUDA is not available. Using CPU.")

if __name__ == "__main__":
    check_devices() 

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch version: 1.11.0a0+gitbc2c6ed

Available devices:
CUDA is available!
Number of CUDA devices: 1
Device 0: NVIDIA Tegra X1

Current CUDA device: 0
Current device name: NVIDIA Tegra X1


In [ ]:
!python3.8 export.py --weights best.pt --include engine --half --workspace 1 --verbose --opset 12 --device 0

export: data=data/coco128.yaml, weights=['best.pt'], imgsz=[640, 640], batch_size=1, device=0, half=True, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, cache=, simplify=False, mlmodel=False, opset=12, verbose=True, workspace=1, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['engine']
YOLOv5 🚀 ee6107e Python-3.8.0 torch-1.11.0a0+gitbc2c6ed CUDA:0 (NVIDIA Tegra X1, 1980MiB)

Fusing layers... 


In [8]:
import tensorrt as trt
import numpy as np
import cv2
import time

def load_engine(engine_path):
    TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
    with open(engine_path, 'rb') as f, trt.Runtime(TRT_LOGGER) as runtime:
        return runtime.deserialize_cuda_engine(f.read())

def main():
    # 加载 engine 文件
    engine_path = 'best.engine'
    print(f"Loading engine from {engine_path}")
    engine = load_engine(engine_path)
    
    if engine is None:
        print("Failed to load engine!")
        return
    
    print("Engine loaded successfully!")
    print(f"Number of bindings: {engine.num_bindings}")
    
    # 打印输入输出信息
    for i in range(engine.num_bindings):
        name = engine.get_binding_name(i)
        shape = engine.get_binding_shape(i)
        dtype = engine.get_binding_dtype(i)
        print(f"Binding {i}: {name}, Shape: {shape}, Dtype: {dtype}")

if __name__ == "__main__":
    main() 

Loading engine from best.engine
Engine loaded successfully!
Number of bindings: 3
Binding 0: images, Shape: (1, 3, 640, 640), Dtype: DataType.HALF
Binding 1: output1, Shape: (1, 32, 160, 160), Dtype: DataType.HALF
Binding 2: output0, Shape: (1, 25200, 40), Dtype: DataType.HALF
